In [1]:
import os
import zipfile
import pandas as pd
import basedosdados as bd

INPUT = os.path.join(os.getcwd(), "input")
OUTPUT = os.path.join(os.getcwd(), "output")

os.makedirs(INPUT, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)

In [2]:
def read_sheet(sheet_name: str, skiprows: int = 3) -> pd.DataFrame:
    return pd.read_excel(
        os.path.join(
            INPUT,
            "TDI_-_Alunos_com_defici_ncia.xlsx"
        ),
        skiprows=skiprows,
        sheet_name=sheet_name
    )

In [3]:
excel_data = pd.ExcelFile(os.path.join(
            INPUT,
            "TDI_-_Alunos_com_defici_ncia.xlsx"
        ))

# Get the sheet names
sheet_names = excel_data.sheet_names

In [4]:
dfs = {
    sheet_name: read_sheet(sheet_name)
    for sheet_name in sheet_names
}

In [5]:
dfs

{'2007':     Unnamed: 0                                         Unnamed: 1  \
 0          NaN                                                  0   
 1          NaN                                                  1   
 2          NaN                                                  2   
 3          NaN                                                  3   
 4          NaN                                                  4   
 5          NaN                                                  5   
 6          NaN                                                 11   
 7          NaN                                                 12   
 8          NaN                                                 13   
 9          NaN                                                 14   
 10         NaN                                                 15   
 11         NaN                                                 16   
 12         NaN                                                 17   
 13         

In [6]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()  # Adds a blank line for readability


Sheet: 2007
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Distorção Idade-Série',
       'Distorção Idade-Série.1', 'Distorção Idade-Série.2'],
      dtype='object')

Sheet: 2008
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Distorção Idade-Série',
       'Distorção Idade-Série.1', 'Distorção Idade-Série.2'],
      dtype='object')

Sheet: 2009
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Distorção Idade-Série',
       'Distorção Idade-Série.1', 'Distorção Idade-Série.2'],
      dtype='object')

Sheet: 2010
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Distorção Idade-Série',
       'Distorção Idade-Série.1', 'Distorção Idade-Série.2'],
      dtype='object')

Sheet: 2011
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Distorção Idade-Série',
       'Distorção Idade-Série.1', 'Distorção Idade-Série.2'],
      dtype='object')

Sheet: 2012
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Distorção Idade-Série',
       'Distorção Idade-Série.1', 'Distorção Idade-Série.2'

In [7]:
dataframes = {}

for table_name, columns in dfs.items():
    df = pd.DataFrame(columns)  # Create DataFrame for each table
    dataframes[table_name] = df  # Store the DataFrame in a dictionary



In [8]:
RENAME_COLUMNS = {
    'Unnamed: 1':'id_uf',
    'Unnamed: 2':'nome',
    'Distorção Idade-Série': 'Ensino Fundamental – Anos Iniciais',
    'Distorção Idade-Série.1':'Ensino Fundamental – Anos Finais',
    'Distorção Idade-Série.2':'Ensino Médio Regular'   
}

In [9]:
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols_drop = [
        col
        for col in df.columns
        if col.startswith("Unnamed")
    ]

    return df.drop(columns=cols_drop)

dfs = {
    name: drop_unused_columns(
        df.rename(columns=RENAME_COLUMNS, errors="raise")
    )
    for name, df in dfs.items()
}

In [10]:
dfs

{'2007':                                                 id_uf                 nome  \
 0                                                   0               Brasil   
 1                                                   1                Norte   
 2                                                   2             Nordeste   
 3                                                   3              Sudeste   
 4                                                   4                  Sul   
 5                                                   5         Centro-Oeste   
 6                                                  11             Rondônia   
 7                                                  12                 Acre   
 8                                                  13             Amazonas   
 9                                                  14              Roraima   
 10                                                 15                 Pará   
 11                                         

In [11]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()  # Adds a blank line for readability

Sheet: 2007
Index(['id_uf', 'nome', 'Ensino Fundamental – Anos Iniciais',
       'Ensino Fundamental – Anos Finais', 'Ensino Médio Regular'],
      dtype='object')

Sheet: 2008
Index(['id_uf', 'nome', 'Ensino Fundamental – Anos Iniciais',
       'Ensino Fundamental – Anos Finais', 'Ensino Médio Regular'],
      dtype='object')

Sheet: 2009
Index(['id_uf', 'nome', 'Ensino Fundamental – Anos Iniciais',
       'Ensino Fundamental – Anos Finais', 'Ensino Médio Regular'],
      dtype='object')

Sheet: 2010
Index(['id_uf', 'nome', 'Ensino Fundamental – Anos Iniciais',
       'Ensino Fundamental – Anos Finais', 'Ensino Médio Regular'],
      dtype='object')

Sheet: 2011
Index(['id_uf', 'nome', 'Ensino Fundamental – Anos Iniciais',
       'Ensino Fundamental – Anos Finais', 'Ensino Médio Regular'],
      dtype='object')

Sheet: 2012
Index(['id_uf', 'nome', 'Ensino Fundamental – Anos Iniciais',
       'Ensino Fundamental – Anos Finais', 'Ensino Médio Regular'],
      dtype='object')

Sheet: 201

In [12]:
melted_dataframe = pd.concat(
    [
        df.pipe(
            lambda d: d.loc[(d["id_uf"].astype(str).str.len() == 2)]
        )
        .pipe(
            lambda d: pd.melt(
                d,
                id_vars=["id_uf", "nome"],
                value_vars=d.columns.difference(["id_uf", "nome"]).tolist(),  # Convert to list
                var_name="etapa_ensino",
                value_name="tdi",
            )
        )
        .assign(ano=ano)
        for ano, df in dfs.items()
    ]
)

In [13]:
melted_dataframe

,id_uf,nome,etapa_ensino,tdi,ano
0,11,Rondônia,Ensino Fundamental – Anos Finais,65.8,2007
1,12,Acre,Ensino Fundamental – Anos Finais,60.0,2007
2,13,Amazonas,Ensino Fundamental – Anos Finais,75.7,2007
3,14,Roraima,Ensino Fundamental – Anos Finais,69.5,2007
4,15,Pará,Ensino Fundamental – Anos Finais,68.4,2007
...,...,...,...,...,...
76,43,Rio Grande do Sul,Ensino Médio Regular,53.2,2021
77,50,Mato Grosso do Sul,Ensino Médio Regular,57.9,2021
78,51,Mato Grosso,Ensino Médio Regular,40.7,2021
79,52,Goiás,Ensino Médio Regular,46.5,2021


In [14]:
melted_dataframe['id_uf'].unique()

array([11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31,
       32, 33, 35, 41, 42, 43, 50, 51, 52, 53], dtype=object)

In [15]:
bd_dir = pd.read_csv(
    '/home/vilelaluiza/dados/br_inep_sinopse_estatistica_educacao_basica/input/br_bd_diretorios_brasil.uf.csv'
)

melted_dataframe["nome"] = (
    melted_dataframe["nome"]
    .astype(str)
    .replace(
        {i["nome"]: i["sigla"] for i in bd_dir.to_dict("records")}  # type: ignore
    )
)

melted_dataframe = melted_dataframe.rename(
    columns={"nome": "sigla_uf"}, errors="raise"
)

In [16]:
melted_dataframe = melted_dataframe[
    [
        "ano",
        "sigla_uf",
        "etapa_ensino",
        "tdi",
    ]
]

In [17]:
melted_dataframe

,ano,sigla_uf,etapa_ensino,tdi
0,2007,RO,Ensino Fundamental – Anos Finais,65.8
1,2007,AC,Ensino Fundamental – Anos Finais,60.0
2,2007,AM,Ensino Fundamental – Anos Finais,75.7
3,2007,RR,Ensino Fundamental – Anos Finais,69.5
4,2007,PA,Ensino Fundamental – Anos Finais,68.4
...,...,...,...,...
76,2021,RS,Ensino Médio Regular,53.2
77,2021,MS,Ensino Médio Regular,57.9
78,2021,MT,Ensino Médio Regular,40.7
79,2021,GO,Ensino Médio Regular,46.5


In [18]:
path = os.path.join(
        OUTPUT, "educacao_especial_distorcao_idade_serie"
    )

os.makedirs(path, exist_ok=True)
melted_dataframe.astype(str).to_csv(os.path.join(path, "data.csv"), index=False)